In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Load dataset
file_path = 'test.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(df.head())

print(df.tail())

# Display the summary of the dataset
print(df.info())


   Id              Time              Machine Component            Parameter  \
0   1  01-05-2022 00:00     Backhoe_Loader_1      Misc       System Voltage   
1   2  01-05-2022 00:10  Articulated_Truck_1      Misc  Air Filter Pressure   
2   3  01-05-2022 00:20              Dozer_1     Drive         Pedal Sensor   
3   4  01-05-2022 00:30     Backhoe_Loader_1    Engine         Oil Pressure   
4   5  01-05-2022 00:40     Backhoe_Loader_1    Engine         Oil Pressure   

    Value  
0   85.49  
1   19.73  
2   16.59  
3  142.67  
4  141.78  
          Id              Time              Machine Component  \
29995  29996  25-11-2022 07:10          Excavator_1      Misc   
29996  29997  25-11-2022 07:20          Excavator_1      Fuel   
29997  29998  25-11-2022 07:30  Articulated_Truck_1      Misc   
29998  29999  25-11-2022 07:40     Backhoe_Loader_1    Engine   
29999  30000  25-11-2022 07:50              Dozer_1     Drive   

                 Parameter    Value  
29995  Air Filter Pressu

In [ ]:
# Create a function to merge component and parameter into one column
def merge_component_parameter(row):
    component = row['Component']
    parameter = row['Parameter']
    return f"{component}_{parameter}"

# Apply the function to create a new column
df['component_parameter'] = df.apply(merge_component_parameter, axis=1)
df['Time'] = pd.to_datetime(df['Time'], format='%d-%m-%Y %H:%M')
# Pivot the dataframe to create separate columns for each component-parameter combination
df_pivot = df.pivot_table(index=['Time', 'Machine'],
                          columns='component_parameter',
                          values='Value',
                          fill_value=0)

# Reset index to flatten the dataframe
df_pivot.reset_index(inplace=True)


df_pivot.to_csv('transformed_dataset.csv', index=False)

df_pivot

component_parameter,Time,Machine,Drive_Brake Control,Drive_Pedal Sensor,Drive_Transmission Pressure,Engine_Oil Pressure,Engine_Speed,Engine_Temperature,Fuel_Fuel Pressure,Fuel_Temperature,Fuel_Water in Fuel,Misc_Air Filter Pressure,Misc_Exhaust Gas Temperature,Misc_Hydraulic Pump Rate,Misc_System Voltage
0,2022-05-01 00:00:00,Backhoe_Loader_1,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,85.49
1,2022-05-01 00:10:00,Articulated_Truck_1,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,19.73,0.0,0.00,0.00
2,2022-05-01 00:20:00,Dozer_1,0.00,16.59,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00
3,2022-05-01 00:30:00,Backhoe_Loader_1,0.00,0.00,0.0,142.67,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00
4,2022-05-01 00:40:00,Backhoe_Loader_1,0.00,0.00,0.0,141.78,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2022-11-25 07:10:00,Excavator_1,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,15.16,0.0,0.00,0.00
29996,2022-11-25 07:20:00,Excavator_1,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,4575.83,0.00,0.0,0.00,0.00
29997,2022-11-25 07:30:00,Articulated_Truck_1,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,157.37,0.00
29998,2022-11-25 07:40:00,Backhoe_Loader_1,0.00,0.00,0.0,0.00,3258.62,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00


In [ ]:
lst=list(df['component_parameter'].unique())
print(lst)
df = df.sort_values(by='Time')
df.set_index('Time',inplace=True)

['Misc_System Voltage', 'Misc_Air Filter Pressure', 'Drive_Pedal Sensor', 'Engine_Oil Pressure', 'Misc_Hydraulic Pump Rate', 'Drive_Brake Control', 'Fuel_Water in Fuel', 'Fuel_Fuel Pressure', 'Engine_Temperature', 'Fuel_Temperature', 'Misc_Exhaust Gas Temperature', 'Drive_Transmission Pressure', 'Engine_Speed']


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt


# # Filter data for a specific machine and component
# machine_name = 'Backhoe_Loader_1'  # Replace with the actual machine name
# component_name = 'Drive_PedalSensor'  # Replace with the actual component name

# filtered_data = df[(df['Machine'] == machine_name) & (df['Component'] == component_name)]

# # Plotting the data
# plt.figure(figsize=(12, 6))

# # Plotting the trend of 'ParameterValue' over time for the specific machine and component
# plt.plot(filtered_data['Time'], filtered_data['Value'], label=f'{component_name} Value', color='b')

# plt.xlabel('DateTime')
# plt.ylabel('Parameter Value')
# plt.title(f'Trend of {component_name} Value Over Time for {machine_name}')
# plt.legend()
# plt.grid(True)
# plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
# # plt.tight_layout()  # Adjust layout to prevent clipping of labels
# plt.show()


In [ ]:
machine='Backhoe_Loader_1'  #input here
df=df[df['Machine']==machine]
df

,Id,Machine,Component,Parameter,Value,component_parameter
Time,,,,,,
2022-05-01 00:00:00,1,Backhoe_Loader_1,Misc,System Voltage,85.49,Misc_System Voltage
2022-05-01 00:30:00,4,Backhoe_Loader_1,Engine,Oil Pressure,142.67,Engine_Oil Pressure
2022-05-01 00:40:00,5,Backhoe_Loader_1,Engine,Oil Pressure,141.78,Engine_Oil Pressure
2022-05-01 01:40:00,11,Backhoe_Loader_1,Engine,Temperature,642.51,Engine_Temperature
2022-05-01 01:50:00,12,Backhoe_Loader_1,Misc,System Voltage,7.42,Misc_System Voltage
...,...,...,...,...,...,...
2022-11-25 04:40:00,29981,Backhoe_Loader_1,Fuel,Fuel Pressure,79.83,Fuel_Fuel Pressure
2022-11-25 04:50:00,29982,Backhoe_Loader_1,Misc,Exhaust Gas Temperature,1767.44,Misc_Exhaust Gas Temperature
2022-11-25 06:20:00,29991,Backhoe_Loader_1,Engine,Oil Pressure,57.74,Engine_Oil Pressure


In [ ]:
l=float('-inf')
h=float('inf')
thresholds = {
    'Engine_Oil Pressure': {'low': 25, 'high': 65, 'probability': 2},
    'Engine_Speed': {'low':l, 'high': 1800, 'probability': 1},
    'Engine_Temparature': {'low':l, 'high': 105, 'probability': 2},
    'Drive_Brake Control': {'low': 1, 'high':h, 'probability': 1},
    'Drive_Transmission Pressure': {'low': 200, 'high': 450, 'probability': 1},
    'Drive_Pedal Sensor': {'low':l, 'high': 4.7, 'probability': 0},
    'Fuel_Water in Fuel': {'low':l, 'high': 1800, 'probability': 2},
    'Fuel_Level': {'low': 1, 'high':h, 'probability': 0},
    'Fuel_Pressure': {'low': 35, 'high': 65, 'probability': 0},
    'Fuel_Temparature': {'low':l, 'high': 400, 'probability': 2},
    'Misc_System Voltage': {'low': 12.0, 'high': 15.0, 'probability': 2},
    'Misc_Exhaust Gas Temparature': {'low':l, 'high': 365, 'probability': 2},
    'Misc_Hydraulic Pump Rate': {'low':l, 'high': 125, 'probability': 1},
    'Misc_Air Filter Pressure': {'low': 20,'high':h, 'probability': 1}
}

component_parameter_mapping = {
    'Engine': ['Engine_Oil Pressure', 'Engine_Speed', 'Engine_Temparature'],
    'Misc': ['Misc_Air Filter Pressure', 'Misc_Exhaust Gas Temparature', 'Misc_Hydraulic Pump Rate', 'Misc_System Voltage'],
    'Drive': ['Drive_Brake Control', 'Drive_Pedal Sensor', 'Drive_Transmission Pressure'],
    'Fuel': ['Fuel_Level', 'Fuel_Pressure', 'Fuel_Temparature', 'Fuel_Water in Fuel']
}

probability={
    0:'Low',
    1:'Medium',
    2:'High'
}

component='Drive' #input here

In [ ]:
df.shape

(7498, 6)

In [ ]:
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data.iloc[i:(i + seq_length)].values
        y = data.iloc[i + seq_length].values
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [ ]:
def predict_future(model, data, seq_length, steps_ahead):
    future_predictions = []
    current_seq = data[-seq_length:].astype(np.float32)  # Last sequence from the data

    for _ in range(steps_ahead):
        prediction = model.predict(current_seq[np.newaxis, :, :])[0]
        future_predictions.append(prediction)
        current_seq = np.append(current_seq[1:], [prediction], axis=0)  # Slide window

    return np.array(future_predictions)

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize failure_dates dictionary
failure_dates = {}

# Iterate through each part in the component parameter mapping
for part in component_parameter_mapping[component]:
    seq_length = 10
    data = df[df['component_parameter'] == part]

    # Drop unnecessary columns
    data = data.drop(columns=['Parameter', 'Machine', 'Id', 'Component', 'component_parameter'])

    # Create sequences for the model
    X, y = create_sequences(data, seq_length)

    # Split data into training and testing sets
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, X_train.shape[2])))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(X_train.shape[2]))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=20, verbose=0)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"RMSE for {part}: {rmse}")

    # Number of steps ahead to predict
    steps_ahead = 50  # Adjust as needed
    time_step_seconds = 60  # Adjust based on your time step interval

    # Predict future values
    future_predictions = predict_future(model, data.values, seq_length, steps_ahead)

    # Check for failure conditions
    current_time = data.index[-1]
    for i in range(steps_ahead):
        if future_predictions[i, 0] > thresholds[part]['high'] or future_predictions[i, 0] < thresholds[part]['low']:
            failure_dates[part] = current_time + pd.Timedelta(seconds=(i + 1) * time_step_seconds)
            break

# Display the failure dates and probability of total failure
prob = []
if not failure_dates:
    print("No failures predicted within the specified horizon.")
else:
    for param in component_parameter_mapping[component]:
        if param in failure_dates:
            print(f"The predicted failure date for {param} is: {failure_dates[param]} and the probability is: {probability[thresholds[param]['probability']]}")
            prob.append(thresholds[param]['probability'])
        else:
            print(f"No failure predicted for {param}")

    print("\nProbability of Total Failure: ", probability[max(prob)])


4/4 [==============================] - 3s 10ms/step
RMSE for Drive_Brake Control: 1.1290367562717627
4/4 [==============================] - 1s 6ms/step
RMSE for Drive_Pedal Sensor: 8.63197172686369
4/4 [==============================] - 1s 7ms/step
RMSE for Drive_Transmission Pressure: 560.5512831080973
1/1 [==============================] - 0s 27ms/step
No failure predicted for Drive_Brake Control
The predicted failure date for Drive_Pedal Sensor is: 2022-11-23 21:11:00 and the probability is: Low
The predicted failure date for Drive_Transmission Pressure is: 2022-11-24 07:41:00 and the probability is: Medium

Probability of Total Failure:  Medium


In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score, recall_score, f1_score
import numpy as np

# Initialize failure_dates dictionary
failure_dates = {}

# Iterate through each part in the component parameter mapping
for part in component_parameter_mapping[component]:
    seq_length = 10
    data = df[df['component_parameter'] == part]

    # Drop unnecessary columns
    data = data.drop(columns=['Parameter', 'Machine', 'Id', 'Component', 'component_parameter'])

    # Create sequences for the model
    X, y = create_sequences(data, seq_length)

    # Split data into training and testing sets
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, X_train.shape[2])))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(X_train.shape[2]))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, batch_size=1, epochs=20, verbose=0)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print(f"RMSE for {part}: {rmse}")

    # Convert the regression output to binary classification based on thresholds
    y_test_class = (y_test[:, 0] > thresholds[part]['high']) | (y_test[:, 0] < thresholds[part]['low'])
    y_pred_class = (y_pred[:, 0] > thresholds[part]['high']) | (y_pred[:, 0] < thresholds[part]['low'])

    # Calculate classification metrics
    accuracy = accuracy_score(y_test_class, y_pred_class)
    recall = recall_score(y_test_class, y_pred_class)
    f1 = f1_score(y_test_class, y_pred_class)

    print(f"Accuracy for {part}: {accuracy}")
    print(f"Recall for {part}: {recall}")
    print(f"F1 Score for {part}: {f1}")

    # Number of steps ahead to predict
    steps_ahead = 50  # Adjust as needed
    time_step_seconds = 60  # Adjust based on your time step interval

    # Predict future values
    future_predictions = predict_future(model, data.values, seq_length, steps_ahead)

    # Check for failure conditions
    current_time = data.index[-1]
    for i in range(steps_ahead):
        if future_predictions[i, 0] > thresholds[part]['high'] or future_predictions[i, 0] < thresholds[part]['low']:
            failure_dates[part] = current_time + pd.Timedelta(seconds=(i + 1) * time_step_seconds)
            break

# Display the failure dates and probability of total failure
prob = []
if not failure_dates:
    print("No failures predicted within the specified horizon.")
else:
    for param in component_parameter_mapping[component]:
        if param in failure_dates:
            print(f"The predicted failure date for {param} is: {failure_dates[param]} and the probability is: {probability[thresholds[param]['probability']]}")
            prob.append(thresholds[param]['probability'])
        else:
            print(f"No failure predicted for {param}")

    print("\nProbability of Total Failure: ", probability[max(prob)])


4/4 [==============================] - 1s 6ms/step
RMSE for Drive_Brake Control: 1.1208453368485716
Accuracy for Drive_Brake Control: 0.825
Recall for Drive_Brake Control: 0.0
F1 Score for Drive_Brake Control: 0.0
4/4 [==============================] - 1s 6ms/step
RMSE for Drive_Pedal Sensor: 8.395178286691822
Accuracy for Drive_Pedal Sensor: 0.8053097345132744
Recall for Drive_Pedal Sensor: 1.0
F1 Score for Drive_Pedal Sensor: 0.892156862745098
4/4 [==============================] - 1s 11ms/step
RMSE for Drive_Transmission Pressure: 560.9457708224649
Accuracy for Drive_Transmission Pressure: 0.8934426229508197
Recall for Drive_Transmission Pressure: 1.0
F1 Score for Drive_Transmission Pressure: 0.9437229437229437
1/1 [==============================] - 0s 26ms/step
No failure predicted for Drive_Brake Control
The predicted failure date for Drive_Pedal Sensor is: 2022-11-23 21:11:00 and the probability is: Low
The predicted failure date for Drive_Transmission Pressure is: 2022-11-24 07:

In [ ]:

failure_dates = {}
for part in component_parameter_mapping[component]:
  seq_length = 10
  data=df[df['component_parameter']==part]

  data=data.drop(columns=['Parameter','Machine','Id','Component','component_parameter'])

  X, y = create_sequences(data, seq_length)

  split = int(0.8 * len(X))
  X_train, X_test = X[:split], X[split:]
  y_train, y_test = y[:split], y[split:]

  model = Sequential()
  model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, X_train.shape[2])))
  model.add(LSTM(50, return_sequences=False))
  model.add(Dense(25))
  model.add(Dense(X_train.shape[2]))

  # Compile the model
  model.compile(optimizer='adam', loss='mean_squared_error')

  # Train the model
  model.fit(X_train, y_train, batch_size=1, epochs=20,verbose=0)
  # Number of steps ahead to predict
  steps_ahead = 50  # Adjust as needed
  time_step_seconds = 60  # Adjust based on your time step interval

  future_predictions = predict_future(model, data.values, seq_length, steps_ahead)


  # Check for failure conditions

  current_time = data.index[-1]

  for i in range(steps_ahead):
      if future_predictions[i, 0] > thresholds[part]['high'] or future_predictions[i, 0] < thresholds[part]['low']:
          failure_dates[part] = current_time + pd.Timedelta(seconds=(i+1) * time_step_seconds)
          break

  # Display the failure dates
prob=[]
if not failure_dates:
  print("No failures predicted within the specified horizon.")
else:
  for param in component_parameter_mapping[component]:
    if param in failure_dates:
      print(f"The predicted failure date for {param} is: {failure_dates[param]} and the probability is: {probability[thresholds[param]['probability']]}")
      prob.append(thresholds[param]['probability'])
    else:
      print(f"No failure predicted for {param}")


  print("\nProbability of Total Failure: ",probability[max(prob)])

1/1 [==============================] - 0s 16ms/step


IndexError: tuple index out of range

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score

predictions = []
actuals = []

# Assuming you have actual failure dates for evaluation
actual_failure_dates = {
    # Replace with your actual failure dates
    'param1': pd.Timestamp('2022-11-25'),
    'param2': pd.Timestamp('2022-11-08'),
}

for param in component_parameter_mapping[component]:
    # Collect predictions
    if param in failure_dates:
        predictions.append(1)  # 1 indicates a predicted failure
    else:
        predictions.append(0)  # 0 indicates no predicted failure

    # Collect actual outcomes
    if param in actual_failure_dates:
        actuals.append(1)  # 1 indicates an actual failure
    else:
        actuals.append(0)  # 0 indicates no actual failure

# Calculate metrics
accuracy = accuracy_score(actuals, predictions)
f1 = f1_score(actuals, predictions)
recall = recall_score(actuals, predictions)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
